In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import numpy as np
import bidi.algorithm
import arabic_reshaper
import adhtools.utils

In [31]:
fpath = '/media/sf_VBox_Shared/Arabic/Analyses/Fiqh_final/topicmodelling/'

In [38]:
document_topics = pd.read_csv(os.path.join(fpath, 'fiqh_light10_document_topics_20.csv'), index_col=0).sort_index()
df_topics = pd.read_csv(os.path.join(fpath, 'fiqh_light10_topics_20.csv'), index_col=0)
metadata = pd.read_csv(os.path.join(fpath, 'metadata-chapters.csv'), index_col=0)

In [39]:
df_topics.head()

,0,1,2,3,4,5,6,7,8,9
0,عقد,فل,عد,ولد,اذ,مهر,ام,نكاح,ول,زوج
1,اخذ,اذ,وقف,خمس,عشر,حول,صدق,ول,مال,زكا
2,سجد,سلام,ركع,وقت,اول,صل,ول,اذ,امام,صلا
3,امام,فل,اهل,قطع,اسلام,اذ,قتل,حد,مسلم,ول
4,خيار,مال,بااع,عقد,ول,قبض,اذ,ثمن,مشتر,بيع


In [34]:
document_topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0179MalikIbnAnas.Muwatta-00000.xml,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.220000,0.020000,0.020000,0.020000,0.220000,0.220000,0.020000,0.020000
0179MalikIbnAnas.Muwatta-00001.xml,0.000258,0.000258,0.000258,0.000258,0.000258,0.015762,0.000258,0.000258,0.000258,0.000258,0.000258,0.000258,0.633333,0.000258,0.000258,0.000258,0.000258,0.346512,0.000258,0.000258
0179MalikIbnAnas.Muwatta-00002.xml,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.001818,0.020000,0.001818,0.001818,0.001818,0.765455,0.001818,0.001818,0.001818,0.038182,0.147273,0.001818,0.001818
0179MalikIbnAnas.Muwatta-00003.xml,0.001724,0.001724,0.001724,0.018966,0.001724,0.001724,0.001724,0.001724,0.001724,0.018966,0.001724,0.001724,0.536207,0.001724,0.001724,0.001724,0.001724,0.398276,0.001724,0.001724
0179MalikIbnAnas.Muwatta-00004.xml,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.000680,0.531293,0.000680,0.000680,0.000680,0.000680,0.456463,0.000680,0.000680


In [35]:
def get_page_for_doc(fname):
    base_url = 'http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs'
    url = base_url + '?filter=fromInputFile:(*{})'.format(fname)
    return url

In [37]:
for topic_nr in document_topics.columns:
    print('Topic {}'.format(topic_nr))
    print(' '.join(df_topics.loc[int(topic_nr),:]))
    important_docs = document_topics[topic_nr].sort_values(ascending=False).head(10).index
    for doc in important_docs:
        print(get_page_for_doc(doc))
    print('-------\n')

Topic 0
عقد فل عد ولد اذ مهر ام نكاح ول زوج
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0587IbnMascudCalaDinKasani.BadaicSanaic-00222.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0428AlQuduri.Mukhtasar-00044.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0428AlQuduri.Mukhtasar-00041.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0463IbnCabdBarr.KafiFiFiqh-00083.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0460ShaykhTusi.Mabsut-00064.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0334IbnHusaynKhiraqi.MukhtasarMinMasailIbnHanbal-00035.xml)
http://acc.arabic-dh.hum.uu.nl/corpus-frontend/Fiqh-chapters/search/docs?filter=fromInputFile:(*0676Nawawi.ManahijTalibin-00031.xm